# Lab 3: Extending Logistic Regression

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression as SKLLogisticRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn import metrics as mt
from scipy.special import expit
from scipy.optimize import fmin_bfgs
from numpy.linalg import pinv
from numpy import ma
import random
import time
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)
warnings.simplefilter('ignore', ConvergenceWarning)
pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'numpy'

# Business Understanding

Dataset Link: https://www.kaggle.com/datasets/oles04/bundesliga-seasons

# Cleaning Data

In [2]:
df_init = pd.read_csv("../Datasets/bulidata.csv")

df_init.head()

,Unnamed: 0,MATCH_DATE,LEAGUE_NAME,SEASON,LEAGUE,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,HOME_TEAM_ID,HOME_TEAM_NAME,HOME_TEAM,HOME_ICON,AWAY_TEAM_ID,AWAY_TEAM_NAME,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY
0,0,2005-08-05 20:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,München,NaN,1. Spieltag,1,40,FC Bayern München,Bayern,https://i.imgur.com/jJEsJrj.png,87,Borussia Mönchengladbach,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0
1,1,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Köln,NaN,1. Spieltag,1,65,1. FC Köln,Köln,https://upload.wikimedia.org/wikipedia/en/thum...,81,1. FSV Mainz 05,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0
2,2,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Duisburg,NaN,1. Spieltag,1,107,MSV Duisburg,Duisburg,https://upload.wikimedia.org/wikipedia/en/c/c8...,16,VfB Stuttgart,Stuttgart,https://i.imgur.com/v0tkpNx.png,1,1,1.0,0.0,0.0
3,3,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Hamburg,NaN,1. Spieltag,1,100,Hamburger SV,HSV,https://upload.wikimedia.org/wikipedia/commons...,79,1. FC Nürnberg,Nürnberg,https://upload.wikimedia.org/wikipedia/commons...,3,0,0.0,1.0,0.0
4,4,2005-08-06 15:30:00,1. Fussball-Bundesliga 2005/2006,2005,bl1,True,Wolfsburg,NaN,1. Spieltag,1,131,VfL Wolfsburg,Wolfsburg,https://i.imgur.com/ucqKV4B.png,7,Borussia Dortmund,BVB,https://upload.wikimedia.org/wikipedia/commons...,2,2,1.0,0.0,0.0


In [3]:
df_init = df_init.drop([
    "Unnamed: 0", "MATCH_DATE", "LEAGUE", "LEAGUE_NAME", "FINISHED", "LOCATION", "VIEWER", "MATCHDAY",
    "HOME_TEAM_NAME", "HOME_TEAM", "HOME_ICON", "AWAY_TEAM_NAME", "AWAY_TEAM", "AWAY_ICON"
], axis = 1).rename(columns = {
    "SEASON": "season",
    "MATCHDAY_NR": "matchday",
    "HOME_TEAM_ID": "home_team",
    "AWAY_TEAM_ID": "away_team",
    "GOALS_HOME": "home_goals",
    "GOALS_AWAY": "away_goals",
    "DRAW": "draw",
    "WIN_HOME": "home_win",
    "WIN_AWAY": "home_loss"
})
df_init[["home_team", "away_team"]] = df_init[["home_team", "away_team"]].astype(str)

df_init.head()

,season,matchday,home_team,away_team,home_goals,away_goals,draw,home_win,home_loss
0,2005,1,40,87,3,0,0.0,1.0,0.0
1,2005,1,65,81,1,0,0.0,1.0,0.0
2,2005,1,107,16,1,1,1.0,0.0,0.0
3,2005,1,100,79,3,0,0.0,1.0,0.0
4,2005,1,131,7,2,2,1.0,0.0,0.0


In [4]:
df_home = df_init.copy().rename(columns = {
    "home_team": "team",
    "away_team": "opponent",
    "home_goals": "scored",
    "away_goals": "conceded",
    "home_win": "win",
    "home_loss": "loss"
})
df_away = df_init.copy().rename(columns = {
    "home_team": "opponent",
    "away_team": "team",
    "home_goals": "conceded",
    "away_goals": "scored",
    "home_win": "loss",
    "home_loss": "win"
})
df_home["home"] = "1"
df_away["home"] = "0"
df_all = pd.concat([df_home, df_away])
df_all = df_all.sort_values(["season", "matchday"])

df_all

,season,matchday,team,opponent,scored,conceded,draw,win,loss,home
0,2005,1,40,87,3,0,0.0,1.0,0.0,1
1,2005,1,65,81,1,0,0.0,1.0,0.0,1
2,2005,1,107,16,1,1,1.0,0.0,0.0,1
3,2005,1,100,79,3,0,0.0,1.0,0.0,1
4,2005,1,131,7,2,2,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
5503,2022,34,40,65,2,1,0.0,1.0,0.0,0
5504,2022,34,9,1635,2,4,0.0,0.0,1.0,0
5505,2022,34,175,16,1,1,1.0,0.0,0.0,0
5506,2022,34,134,80,0,1,0.0,0.0,1.0,0


In [5]:
df_all["scored_5"] = df_all.groupby(["season", "team"])["scored"].transform(lambda x: x.rolling(5).sum().shift())
df_all["conceded_5"] = df_all.groupby(["season", "team"])["conceded"].transform(lambda x: x.rolling(5).sum().shift())
df_all["win_5"] = df_all.groupby(["season", "team"])["win"].transform(lambda x: x.rolling(5).sum().shift())
df_all["draw_5"] = df_all.groupby(["season", "team"])["draw"].transform(lambda x: x.rolling(5).sum().shift())
df_all["loss_5"] = df_all.groupby(["season", "team"])["loss"].transform(lambda x: x.rolling(5).sum().shift())
df_all["game_pts"] = df_all["win"] * 3 + df_all["draw"]
df_all["pts"] = df_all.groupby(["season", "team"])["game_pts"].cumsum().sub(df_all["game_pts"])

df_all["opp_scored_5"] = df_all.groupby(["season", "opponent"])["scored"].transform(lambda x: x.rolling(5).sum().shift())
df_all["opp_conceded_5"] = df_all.groupby(["season", "opponent"])["conceded"].transform(lambda x: x.rolling(5).sum().shift())
df_all["opp_win_5"] = df_all.groupby(["season", "opponent"])["win"].transform(lambda x: x.rolling(5).sum().shift())
df_all["opp_draw_5"] = df_all.groupby(["season", "opponent"])["draw"].transform(lambda x: x.rolling(5).sum().shift())
df_all["opp_loss_5"] = df_all.groupby(["season", "opponent"])["loss"].transform(lambda x: x.rolling(5).sum().shift())
df_all["opp_game_pts"] = df_all["loss"] * 3 + df_all["draw"]
df_all["opp_pts"] = df_all.groupby(["season", "team"])["opp_game_pts"].cumsum().sub(df_all["opp_game_pts"])
df_all["pts_diff"] = df_all["pts"] - df_all["opp_pts"]

df_all

,season,matchday,team,opponent,scored,conceded,draw,win,loss,home,scored_5,conceded_5,win_5,draw_5,loss_5,game_pts,pts,opp_scored_5,opp_conceded_5,opp_win_5,opp_draw_5,opp_loss_5,opp_game_pts,opp_pts,pts_diff
0,2005,1,40,87,3,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,2005,1,65,81,1,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,2005,1,107,16,1,1,1.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
3,2005,1,100,79,3,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,2005,1,131,7,2,2,1.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,2022,34,40,65,2,1,0.0,1.0,0.0,0,12.0,7.0,3.0,0.0,2.0,3.0,68.0,6.0,11.0,1.0,1.0,3.0,0.0,23.0,45.0
5504,2022,34,9,1635,2,4,0.0,0.0,1.0,0,7.0,15.0,2.0,1.0,2.0,0.0,31.0,4.0,7.0,1.0,0.0,4.0,3.0,58.0,-27.0
5505,2022,34,175,16,1,1,1.0,0.0,0.0,0,9.0,9.0,2.0,0.0,3.0,1.0,35.0,6.0,9.0,1.0,2.0,2.0,1.0,59.0,-24.0
5506,2022,34,134,80,0,1,0.0,0.0,1.0,0,8.0,9.0,1.0,1.0,3.0,0.0,36.0,7.0,7.0,2.0,1.0,2.0,3.0,57.0,-21.0


In [6]:
results = []
for i, row in df_all.iterrows():
    if row["draw"] == 1:
        results.append("Draw")
    elif row["win"] == 1:
        results.append("Win")
    else:
        results.append("Loss")
df_all["result"] = results

df_all

,season,matchday,team,opponent,scored,conceded,draw,win,loss,home,scored_5,conceded_5,win_5,draw_5,loss_5,game_pts,pts,opp_scored_5,opp_conceded_5,opp_win_5,opp_draw_5,opp_loss_5,opp_game_pts,opp_pts,pts_diff,result
0,2005,1,40,87,3,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Win
1,2005,1,65,81,1,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Win
2,2005,1,107,16,1,1,1.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,Draw
3,2005,1,100,79,3,0,0.0,1.0,0.0,1,NaN,NaN,NaN,NaN,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Win
4,2005,1,131,7,2,2,1.0,0.0,0.0,1,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,Draw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5503,2022,34,40,65,2,1,0.0,1.0,0.0,0,12.0,7.0,3.0,0.0,2.0,3.0,68.0,6.0,11.0,1.0,1.0,3.0,0.0,23.0,45.0,Win
5504,2022,34,9,1635,2,4,0.0,0.0,1.0,0,7.0,15.0,2.0,1.0,2.0,0.0,31.0,4.0,7.0,1.0,0.0,4.0,3.0,58.0,-27.0,Loss
5505,2022,34,175,16,1,1,1.0,0.0,0.0,0,9.0,9.0,2.0,0.0,3.0,1.0,35.0,6.0,9.0,1.0,2.0,2.0,1.0,59.0,-24.0,Draw
5506,2022,34,134,80,0,1,0.0,0.0,1.0,0,8.0,9.0,1.0,1.0,3.0,0.0,36.0,7.0,7.0,2.0,1.0,2.0,3.0,57.0,-21.0,Loss


In [7]:
df = df_all[df_all["home"] == "1"].dropna().reset_index(drop = True).drop([
    "season", "matchday", "team", "opponent", "scored", "conceded", "draw", "win", "loss", 
    "home", "game_pts", "pts", "opp_game_pts", "opp_pts"
], axis = 1)
df.loc[:, df.columns != "result"] = df.loc[:, df.columns != "result"].astype(int)

df 

,scored_5,conceded_5,win_5,draw_5,loss_5,opp_scored_5,opp_conceded_5,opp_win_5,opp_draw_5,opp_loss_5,pts_diff,result
0,5,8,1,2,2,6,16,0,1,4,-3,Win
1,8,9,1,2,2,8,5,2,2,1,-3,Win
2,3,9,1,1,3,3,14,0,0,5,-6,Loss
3,6,5,1,3,1,8,6,1,3,1,0,Loss
4,11,10,2,1,2,8,9,2,0,3,0,Win
...,...,...,...,...,...,...,...,...,...,...,...,...
4693,11,6,3,1,1,7,12,2,0,3,-3,Loss
4694,7,4,4,0,1,15,7,2,1,2,33,Win
4695,9,6,2,2,1,9,9,3,0,2,-24,Draw
4696,7,7,2,1,2,9,8,3,1,1,27,Win


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   scored_5        4698 non-null   int64 
 1   conceded_5      4698 non-null   int64 
 2   win_5           4698 non-null   int64 
 3   draw_5          4698 non-null   int64 
 4   loss_5          4698 non-null   int64 
 5   opp_scored_5    4698 non-null   int64 
 6   opp_conceded_5  4698 non-null   int64 
 7   opp_win_5       4698 non-null   int64 
 8   opp_draw_5      4698 non-null   int64 
 9   opp_loss_5      4698 non-null   int64 
 10  pts_diff        4698 non-null   int64 
 11  result          4698 non-null   object
dtypes: int64(11), object(1)
memory usage: 440.6+ KB


In [9]:
df.describe()

,scored_5,conceded_5,win_5,draw_5,loss_5,opp_scored_5,opp_conceded_5,opp_win_5,opp_draw_5,opp_loss_5,pts_diff
count,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000,4698.000000
mean,5.985951,6.200085,1.714985,1.471052,1.813963,5.998936,6.213069,1.718178,1.464666,1.817156,-0.272669
std,3.465548,3.327101,1.166687,1.068902,1.147927,3.273398,3.586271,1.141926,1.073480,1.196647,16.771135
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-63.000000
25%,3.000000,4.000000,1.000000,1.000000,1.000000,3.000000,4.000000,1.000000,1.000000,1.000000,-12.000000
50%,6.000000,6.000000,2.000000,1.000000,2.000000,6.000000,6.000000,2.000000,1.000000,2.000000,0.000000
75%,8.000000,8.000000,2.000000,2.000000,3.000000,8.000000,8.000000,2.000000,2.000000,3.000000,9.000000
max,25.000000,22.000000,5.000000,5.000000,5.000000,19.000000,23.000000,5.000000,5.000000,5.000000,78.000000


In [10]:
pd.Series.value_counts(df["result"]) / len(df)

Win     0.427842
Draw    0.287569
Loss    0.284589
Name: result, dtype: float64

# Modeling

## Logistic Regression Implementation

In [11]:
# from last time, our logistic regression algorithm is given by (including everything we previously had):
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, regularization = "none", mixture = None, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.reg = regularization
        self.mix = mixture
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        return gradient
    
    # Calculate the L1 term (lasso)
    def _get_L1(self):
        return -2 * self.C * np.sign(self.w_[1:])
    
    # Calculate the L2 term (ridge)
    def _get_L2(self):
        return -2 * self.w_[1:] * self.C
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
       
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            
            # Add penalty term
            if self.reg == "l1":
                gradient[1:] += self._get_L1()
            elif self.reg == "l2":
                gradient[1:] += self._get_L2()
            elif self.reg == "both":
                gradient[1:] += (1 - self.mix) * self._get_L2()
                gradient[1:] += self.mix * self._get_L1()
            
            self.w_ += gradient*self.eta # multiply by learning rate 
            # add bacause maximizing 

In [12]:
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        
        return gradient

In [13]:
class HessianLogisticRegression(BinaryLogisticRegression):
    # just overwrite gradient function
    def _get_gradient(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X # calculate the hessian
        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        
        return pinv(hessian) @ gradient

In [14]:
class MyLogisticRegression:
    def __init__(self, eta, iterations=20, solver="steepest", regularization="none", mixture=0.5, C=0.001):
        assert solver in ["steepest", "stochastic", "newton"], "Invalid solver input"
        assert regularization in ["none", "l1", "l2", "both"], "Invalid regularization input"
        assert mixture >= 0 and mixture <= 1, "Invalid mixture"
        
        self.eta = eta
        self.iters = iterations
        self.opt = solver
        self.reg = regularization
        self.mix = mixture
        self.C = C
        
    def __str__(self):
        if(hasattr(self,'w_')):
            # is we have trained the object
            return 'Multiclass Logistic Regression Object with {} solver and {} regularization and coefficients:\n'.format(self.opt, self.reg) + str(self.w_) 
        else:
            return 'Untrained Multiclass Logistic Regression Object with {} solver and {} regularization'.format(self.opt, self.reg)
     
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        # Parameters to pass into binary classifier
        params = {
            "eta": self.eta,
            "iterations": self.iters,
            "regularization": self.reg,
            "mixture": self.mix,
            "C": self.C
        }
        # Set solver based on specified optimization method
        if self.opt == "steepest":
            self.solver = BinaryLogisticRegression
        elif self.opt == "stochastic":
            self.solver = StochasticLogisticRegression
        else:
            self.solver = HessianLogisticRegression
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = self.solver(**params)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
    # Set specified hyperparameters after tuning (or manually)
    def set_hyperparameters(self, params):
        # Set parameters with defined hyperparameters
        if params["solver"] != None:
            assert params["solver"] in ["steepest", "stochastic", "newton"], "Invalid solver input"
            self.opt = params["solver"]
        
        if params["regularization"] != None:
            assert params["regularization"] in ["none", "l1", "l2", "both"], "Invalid regularization input"
            self.reg = params["regularization"]
            
        if params["mixture"] != None:
            assert params["mixture"] >= 0 and params["mixture"] <= 1, "Invalid mixture"
            self.mix = params["mixture"]
            
        if params["penalty"] != None:
            self.C = params["penalty"]

## Model Fitting

### My Implementation

In [15]:
# Create a hyperparameter grid for solver, regularization, mixture, and C
def create_grid(
    combinations=100, seed = None, 
    solver = None, regularization = None, mixture = None, C = None
):
    # Set random seed if defined
    if seed != None:
        random.seed(seed)

    # Create data frame for grid
    grid = pd.DataFrame()

    # Create ranges of values
    opt = ["steepest", "stochastic", "newton"]
    reg = ["none", "l1", "l2", "both"]
    mix = np.linspace(0, 1, 1000)
    pen = np.logspace(-5, 1, 20)

    # Randomly sample values
    if solver == None:
        grid["solver"] = random.choices(opt, k = combinations)
    else:
        grid["solver"] = solver

    if regularization == None:
        grid["regularization"] = random.choices(reg, k = combinations)
    else:
        grid["regularization"] = regularization

    if mixture == None:
        grid["mixture"] = random.choices(mix, k = combinations)
    else:
        grid["mixture"] = mixture

    if C == None:
        grid["penalty"] = random.choices(pen, k = combinations)
    else:
        grid["penalty"] = C

    return grid

In [16]:
# Tune a grid of hyperparameters
def tune(X, y, cv_object, grid, params):
    # Lists to store accuracy and time of each combination
    acc = []
    times = []
    # Iterate through each combination in the grid
    for row in grid.itertuples():
        curr_acc = []
        curr_times = []
        # Iterate through each train/test split in cv_object
        for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
            # Extract hyperparameters
            params["solver"] = row.solver
            params["regularization"] = row.regularization
            params["mixture"] = row.mixture
            params["C"] = row.penalty

            # Fit model with these hyperparameters and calculate the accuracy
            model = MyLogisticRegression(**params)
            start = time.time()
            model.fit(X[train_indices], y[train_indices])
            curr_times.append(time.time() - start)
            yhat = model.predict(X[test_indices])
            curr_acc.append(accuracy_score(y[test_indices], yhat))
            
        # Add the mean accuracy and time from all of the train/test splits to acc
        acc.append(np.mean(curr_acc))
        times.append(np.mean(curr_times))

    # Add accuracy and time column to grid
    grid["accuracy"] = acc
    grid["time"] = times
    # Sort grid by accuracy
    grid = grid.sort_values(["accuracy", "time"], ascending = [False, True])
    # Return grid
    return grid

In [17]:
cv_object = ShuffleSplit(n_splits = 2, test_size = 0.2, random_state = 7324)
scaler = StandardScaler()

X = df.loc[:, df.columns != "result"].to_numpy()
X = scaler.fit_transform(X)
y = df["result"].to_numpy()

In [18]:
my_model = MyLogisticRegression(0.1)
print(my_model)

Untrained Multiclass Logistic Regression Object with steepest solver and none regularization


In [19]:
grid_init = create_grid(200, seed = 7324)
grid_init

,solver,regularization,mixture,penalty
0,newton,both,0.163163,0.000183
1,stochastic,both,0.374374,0.014384
2,newton,l2,0.188188,0.000089
3,steepest,both,0.259259,0.000021
4,newton,l2,0.063063,0.014384
...,...,...,...,...
195,newton,none,0.832833,0.000089
196,newton,none,0.492492,0.263665
197,steepest,both,0.210210,0.029764
198,stochastic,l2,0.000000,0.029764


In [20]:
%%time

params = {
    "eta": 0.1,
    "iterations": 20
}

grid_acc = tune(X, y, cv_object, grid_init, params)
grid_acc

CPU times: user 26min 38s, sys: 6min 7s, total: 32min 45s
Wall time: 2min 55s


,solver,regularization,mixture,penalty,accuracy,time
131,steepest,both,0.027027,0.061585,0.473936,0.013651
101,steepest,l1,0.323323,0.001624,0.473936,0.015019
152,steepest,l1,0.727728,0.003360,0.473404,0.015218
197,steepest,both,0.210210,0.029764,0.472872,0.009368
98,steepest,l1,0.947948,0.006952,0.472872,0.012723
...,...,...,...,...,...,...
52,stochastic,l1,0.474474,4.832930,0.262766,0.001212
199,steepest,both,0.495495,4.832930,0.256383,0.011839
112,steepest,both,0.599600,10.000000,0.253191,0.013126
159,steepest,l1,0.432432,4.832930,0.253191,0.014676


In [21]:
grid_acc.groupby(["solver", "regularization"])["accuracy"].describe()

count      mean           std       min       25%  \
solver     regularization                                                      
newton     both             20.0  0.438351  4.701259e-02  0.306915  0.436037   
           l1               13.0  0.459124  8.465768e-03  0.431915  0.459043   
           l2               19.0  0.455823  1.314507e-02  0.425000  0.452394   
           none             16.0  0.462766  1.146633e-16  0.462766  0.462766   
steepest   both             21.0  0.433156  7.668570e-02  0.252660  0.437234   
           l1               13.0  0.429051  7.639248e-02  0.253191  0.431383   
           l2               13.0  0.472136  4.084828e-04  0.471277  0.472340   
           none             18.0  0.472340  1.142410e-16  0.472340  0.472340   
stochastic both             24.0  0.375776  3.829562e-02  0.295745  0.357048   
           l1               14.0  0.378419  5.132262e-02  0.262766  0.356915   
           l2               17.0  0.399155  2.661838e-02  0.325000  0.390957   
           none             12.0  0.392332  2.964568e-02  0.329255  0.384441   

                                50%       75%       max  
solver     regularization                                
newton     both            0.461702  0.462766  0.462766  
           l1              0.462766  0.462766  0.463298  
           l2              0.462766  0.462766  0.464894  
           none            0.462766  0.462766  0.462766  
steepest   both            0.471277  0.472340  0.473936  
           l1              0.472340  0.472872  0.473936  
           l2              0.472340  0.472340  0.472340  
           none            0.472340  0.472340  0.472340  
stochastic both            0.378723  0.405984  0.434574  
           l1              0.394149  0.407979  0.449468  
           l2              0.400532  0.416489  0.432979  
           none            0.397606  0.404388  0.442021

In [22]:
params = grid_acc.to_dict("records")[0]
my_model.set_hyperparameters(params)
print(my_model)

Untrained Multiclass Logistic Regression Object with steepest solver and both regularization


In [23]:
cv_object = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 7324)

In [24]:
%%time

my_acc = []
conf_mat = None
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    my_model = MyLogisticRegression(0.1)
    my_model.set_hyperparameters(params)
    my_model.fit(X[train_indices], y[train_indices])
    yhat = my_model.predict(X[test_indices])
    my_acc.append(accuracy_score(y[test_indices], yhat))
    conf_mat = mt.confusion_matrix(y[test_indices], yhat)
print('Accuracy of: ', np.mean(my_acc))

Accuracy of:  0.45159574468085106
CPU times: user 934 ms, sys: 1.1 s, total: 2.03 s
Wall time: 179 ms


In [25]:
print(my_model)

conf_mat

Multiclass Logistic Regression Object with steepest solver and both regularization and coefficients:
[[-3.43072647e-01 -4.82453640e-02 -2.33017198e-02 -1.33502381e-02
   2.78625794e-02 -6.20469751e-03 -4.31167523e-02 -4.80504058e-02
  -5.51842426e-05  1.61686502e-02 -1.19147247e-02 -3.05697854e-02]
 [-3.40233713e-01 -3.41086272e-02  2.64671603e-02 -4.08800786e-02
   3.62925794e-04  3.93530811e-02 -2.05966080e-02  7.53201403e-02
  -3.07739626e-02 -1.54701115e-02  4.80860747e-02 -7.50699199e-02]
 [-1.13030807e-01  8.65640111e-02  2.36484347e-04  6.15351203e-02
  -2.93428134e-02 -3.00154896e-02  6.74955284e-02 -2.80481032e-02
   3.88236655e-02  1.35319831e-04 -3.78314536e-02  1.06775608e-01]]


array([[ 52,  74, 156],
       [ 43,  81, 135],
       [ 59,  72, 268]])

### Scikit-Learn Implementation

In [26]:
skl_model = SKLLogisticRegression(solver='liblinear',n_jobs=1, 
                           multi_class='ovr', C = 1/0.001, 
                           penalty='l1',max_iter=100) 
print(skl_model)

LogisticRegression(C=1000.0, multi_class='ovr', n_jobs=1, penalty='l1',
                   solver='liblinear')


In [27]:
%%time

skl_acc = []
conf_mat = None
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    skl_model.fit(X[train_indices], y[train_indices])
    yhat = skl_model.predict(X[test_indices])
    skl_acc.append(accuracy_score(y[test_indices], yhat))
    conf_mat = mt.confusion_matrix(y[test_indices], yhat)
print('Accuracy of: ', np.mean(skl_acc))

Accuracy of:  0.4581914893617022
CPU times: user 434 ms, sys: 2.07 s, total: 2.5 s
Wall time: 241 ms


In [28]:
print(skl_model.coef_)

conf_mat

[[-2.43647869e-01  9.85155129e-02  1.43007933e-01  6.79951062e-02
  -1.20214477e-01 -1.44927233e-01 -1.29677761e-01  3.01485603e-02
   1.39972791e-02  4.20315935e-04 -1.45702878e-01]
 [-1.38511572e-01  2.55404438e-02 -1.34278240e-01 -1.13760744e-01
  -1.13242037e-01 -2.70133427e-01  4.85470134e-01  6.54724650e-02
  -5.62933131e-02 -1.60821317e-01 -2.68972355e-01]
 [ 3.06097350e-01 -9.00568309e-02 -7.51349991e-02 -2.46634617e-02
   1.19359604e-01  3.67017710e-01 -3.41359505e-01 -7.51042277e-02
   5.37880667e-02  1.66357423e-01  3.43983583e-01]]


array([[ 44,  59, 179],
       [ 33,  82, 144],
       [ 57,  52, 290]])

## Deployment

# BFGS

## Implementation

### My Implementation

In [29]:
class BFGSLogisticRegression(BinaryLogisticRegression):
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # 1
        H_inv = np.identity(num_features)
        
        for k in range(self.iters):
            # 2
            grad1 = self._get_gradient(Xb,y)
            p = -H_inv @ grad1
            # 3&4
            s = self.eta * p
            self.w_ += s
            # 5
            v = self._get_gradient(Xb,y) - grad1
            # 7
            H_inv = self._sherman_morris(H_inv, s, v)
            
    # vectorized gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        
        # Add penalty term
        if self.reg == "l1":
            gradient[1:] += self._get_L1()
        elif self.reg == "l2":
            gradient[1:] += self._get_L2()
        elif self.reg == "both":
            gradient[1:] += (1 - self.mix) * self._get_L2()
            gradient[1:] += self.mix * self._get_L1()
        
        return gradient
    
    # Approximate inverse Hessian using Sherman Morris
    def _sherman_morris(self, H_inv, s, v):
#         term1_num = np.outer(s, v.T)
#         term1_denom = s.T @ v
#         # Check if the denominator is close to zero
#         if np.abs(term1_denom) < 1e-10:
# #             print(s)
# #             print(v)
#             # Avoid division by nearly zero
# #             print(term1_num)
# #             print(term1_denom)
#             raise Exception("Term 1 denominator nearly 0")
#             return H_inv
#         term1 = term1_num / term1_denom
        
#         term2_num = np.outer(H_inv @ v, s.T @ H_inv)
#         term2_denom = 1 + v.T @ H_inv @ s
#         # Check if the denominator is close to zero
#         if np.abs(term2_denom) < 1e-10:
#             # Avoid division by nearly zero
#             raise Exception("Term 2 denominator nearly 0")
#             return H_inv
#         term2 = term2_num / term2_denom
        
#         return H_inv + term1 - term2
        
        
        term1_num = (np.outer(s.T, v) + H_inv) @ (s @ s.T)
        term1_denom = np.power(np.outer(s.T, v), 2)
        
        term2_num = (H_inv @ np.outer(v, s.T)) + (s @ v.T @ H_inv)
        term2_denom = np.outer(s.T, v)
        
        term1 = term1_num / term1_denom
        term2 = term2_num / term2_denom
        
        return H_inv + term1 - term2

### SciPy Implementation

In [30]:
class BFGSSciPyLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return -ma.sum(ma.log(g[y==1]))-ma.sum(ma.log(1-g[y==0])) + C*sum(w**2) 
        #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))

### New Multiclass Function

In [31]:
class MyLogisticRegression2:
    def __init__(self, eta, iterations=20, solver="steepest", regularization="none", mixture=0.5, C=0.001):
        assert solver in ["steepest", "stochastic", "newton", "bfgs", "bfgs-scipy"], "Invalid solver input"
        assert regularization in ["none", "l1", "l2", "both"], "Invalid regularization input"
        assert mixture >= 0 and mixture <= 1, "Invalid mixture"
        
        self.eta = eta
        self.iters = iterations
        self.opt = solver
        self.reg = regularization
        self.mix = mixture
        self.C = C
        
    def __str__(self):
        if(hasattr(self,'w_')):
            # is we have trained the object
            return 'Multiclass Logistic Regression Object with {} solver and {} regularization and coefficients:\n'.format(self.opt, self.reg) + str(self.w_) 
        else:
            return 'Untrained Multiclass Logistic Regression Object with {} solver and {} regularization'.format(self.opt, self.reg)
     
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        # Parameters to pass into binary classifier
        params = {
            "eta": self.eta,
            "iterations": self.iters,
            "regularization": self.reg,
            "mixture": self.mix,
            "C": self.C
        }
        # Set solver based on specified optimization method
        if self.opt == "steepest":
            self.solver = BinaryLogisticRegression
        elif self.opt == "stochastic":
            self.solver = StochasticLogisticRegression
        elif self.opt == "newton":
            self.solver = HessianLogisticRegression
        elif self.opt == "bfgs": 
            self.solver = BFGSLogisticRegression
        else:
            self.solver = BFGSSciPyLogisticRegression
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = self.solver(**params)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
    # Set specified hyperparameters after tuning (or manually)
    def set_hyperparameters(self, params):
        # Set parameters with defined hyperparameters
        if params["solver"] != None:
            assert params["solver"] in ["steepest", "stochastic", "newton", "bfgs", "bfgs-scipy"], "Invalid solver input"
            self.opt = params["solver"]
        
        if params["regularization"] != None:
            assert params["regularization"] in ["none", "l1", "l2", "both"], "Invalid regularization input"
            self.reg = params["regularization"]
            
        if params["mixture"] != None:
            assert params["mixture"] >= 0 and params["mixture"] <= 1, "Invalid mixture"
            self.mix = params["mixture"]
            
        if params["penalty"] != None:
            self.C = params["penalty"]

## Model Comparison

### My Implementation

In [32]:
params["solver"] = "bfgs"

In [33]:
%%time

my_acc2 = []
conf_mat = None
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    my_model2 = MyLogisticRegression2(0.1, 2)
    my_model2.set_hyperparameters(params)
    my_model2.fit(X[train_indices], y[train_indices])
    yhat = my_model2.predict(X[test_indices])
    my_acc.append(accuracy_score(y[test_indices], yhat))
    conf_mat = mt.confusion_matrix(y[test_indices], yhat)
print('Accuracy of: ', np.mean(my_acc))

Accuracy of:  0.37026595744680846
CPU times: user 276 ms, sys: 535 ms, total: 810 ms
Wall time: 71.9 ms


In [34]:
print(my_model2)

conf_mat

Multiclass Logistic Regression Object with bfgs solver and both regularization and coefficients:
[[    911.21648942    4242.24353381    7833.98523548    9272.02171445
    -6437.3462065    28534.67048703    5168.95052467    4382.51015683
    90854.8070591    -8864.28443567   11024.89654155    7105.71475838]
 [   1009.54148111    6257.97658051   -6216.37176559    4709.53261431
  -260934.43723144   -4712.05557421   10284.26904355   -3402.88406583
     6468.36456319   11014.26920531   -4367.55296657    3208.39405219]
 [   2934.89046609   -2552.62203549   20787.16798519   -2982.79387078
     6429.04150746    5104.66852156   -3477.90909764    8830.56950788
    -4948.86932321  113025.16211718    5408.65405376   -2255.09595287]]


array([[171,  58,  53],
       [160,  56,  43],
       [248,  81,  70]])

### SciPy Implementation

In [35]:
params["solver"] = "bfgs-scipy"

In [36]:
%%time

my_acc = []
conf_mat = None
for iter_num, (train_indices, test_indices) in enumerate(cv_object.split(X,y)):
    my_model = MyLogisticRegression2(0.1)
    my_model.set_hyperparameters(params)
    my_model.fit(X[train_indices], y[train_indices])
    yhat = my_model.predict(X[test_indices])
    my_acc.append(accuracy_score(y[test_indices], yhat))
    conf_mat = mt.confusion_matrix(y[test_indices], yhat)
print('Accuracy of: ', np.mean(my_acc))

Accuracy of:  0.4528723404255318
CPU times: user 5.14 s, sys: 3.86 s, total: 9 s
Wall time: 792 ms


In [37]:
print(my_model)

conf_mat

Multiclass Logistic Regression Object with bfgs-scipy solver and both regularization and coefficients:
[[-9.22510912e-01 -9.27070424e-02 -4.22362305e-02 -1.43705594e-02
   4.56804688e-02 -2.79303469e-02 -9.54659712e-02 -9.56160368e-02
   8.07949002e-04  1.93618604e-02 -1.81400114e-02 -7.81430351e-02]
 [-9.46336835e-01 -7.08650087e-02  2.89310359e-02 -5.94321722e-02
   7.78903975e-03  5.31506450e-02 -4.76405566e-02  1.64097408e-01
  -4.85243635e-02 -2.71863674e-02  7.06935819e-02 -1.55065597e-01]
 [-2.88573920e-01  1.38739868e-01  1.30004714e-02  5.77171586e-02
  -4.21998866e-02 -1.93656438e-02  1.21778246e-01 -6.88853597e-02
   4.35878932e-02  1.08281960e-02 -5.13083696e-02  1.98832267e-01]]


array([[ 33,  48, 201],
       [ 22,  52, 185],
       [ 37,  45, 317]])